## LAB 3. Hugo Cordoba Leal

**1. Define en formato JSON los formatos de datos que espera el sistema, tanto para los datos de entrada a partir de la ‘query’ del usuario como para los datos de salida para la respuesta.**

**Los datos de entrada** que el sistema espera es la 'Query del Usuario'

El sistema espera recibir una query de usuario en un formato que incluye una descripción textual de lo que el usuario está buscando. La estructura esperada es:

In [ ]:
{
    "query": "El usuario busca una película dramática basada en hechos reales sobre la lucha de una madre por liberar a su hijo de una prisión infame."
},

**Datos de Salida: Respuesta del Sistema**
La respuesta del sistema tras procesar la query del usuario consiste en una lista de documentos que representan películas que coinciden con la búsqueda. Un ejemplo del formato JSON de la salida podría ser:

In [ ]:
{
    "retrieved_documents": [
        {
            "movie_id": 100554,
            "title_es": "La corresponsal",
            "duration_mins": 105,
            "country": "Estados Unidos",
            "genre_tags": "Bélico; Acción; Drama; Biográfico; Periodismo; Guerra de Siria; Basado en hechos reales",
            "synopsis": "Marie Colvin (Rosamund Pike) era una periodista reconocida mundialmente por su trabajo en distintos conflictos bélicos. Testigo de algunas cruentas batallas recientes, especialmente de lo sucedido en Oriente Medio, contaba con el respeto tanto de los lectores como de sus compañeros de profesión por su enorme valentía y humildad.",
        },
    ],
}

**2. Reorganiza el flujo de trabajo para que se componga de tres funciones principales: preprocesado, ‘retrieval’ y posprocesado. Estas tres funciones compondrán el flujo de inferencia utilizando los modelos o enfoques almacenados en MLFlow que mejores resultados han obtenido.**

**Preprocesado**
La función de preprocesado preparará la entrada (query del usuario) para que sea adecuada para el proceso de retrieval. Este preprocesamiento incluirá limpiar y estandarizar el texto, extrayendo y transformando los datos necesarios para que el sistema de búsqueda pueda operar de manera óptima. En este caso utilizamos la funcion 'clean_query_txt()'

In [ ]:
def clean_query_txt(query: str) -> str:
    query = query.replace("El usuario busca ", "").strip()
    return query

**Retrieval**
La función de retrieval utilizará el modelo o el enfoque (como embeddings y FAISS) que se ha identificado como el más efectivo en MLFlow. Esta función buscará en la base de datos o índice de películas para encontrar las coincidencias más relevantes basadas en la entrada procesada. Para este caso primero utilizamos una funcione de limpieza de texto 'get_synopsys_txt_2()' y luego utilizamos 'create_docs_to_embedd()'

In [ ]:
def create_docs_to_embedd(
    movies: list[Movie], config: config.RetrievalExpsConfig
) -> list[Document]:
    """
    Convierte una lista de objetos `Movie` a una lista the objetos `Document`
    (usada por Langchain). En esta función se decide que parte de los datos
    será usado como embeddings y que parte como metadata.
    """
    movies_as_docs = []
    for movie in movies:
        content = config.text_to_embed_fn(movie)
        metadata = movie.model_dump()
        doc = Document(page_content=content, metadata=metadata)
        movies_as_docs.append(doc)

    return movies_as_docs

def get_synopsys_txt_2(movie: Movie,) -> str:
    # Pre-limpieza
    synopsis_cleaned = re.sub(r'\(filmaffinity\)', '', movie.synopsis.lower())    
    # Preparar el texto completo
    pre_cleaned = f"{movie.title_es}. {synopsis_cleaned} {movie.genre_tags}"
    clean_movie = re.sub(r';', ',', pre_cleaned.lower())
    return clean_movie

**Posprocesado**
Una vez que se han recuperado los documentos, la función de posprocesado se encargará de formatear adecuadamente estos documentos para su presentación al usuario. Esto puede incluir ordenar los resultados, extraer información relevante de los metadatos y presentarla en un formato amigable para el usuario. En este caso, se analizan los datos para enviarlos a MLFow y poder ver que tan buenos o malos son, segun diferentes metricas como el MRR.

**3. Por último, describe qué métricas técnicas y KPIs de negocio serían interesante monitorizar para asegurar el correcto funcionamiento de la solución.**

Algunas de las métricas y KPIs más relevantes para tener en cuenta son:

/ Métricas Técnicas

- Mean Reciprocal Rank (MRR): Esta métrica es fundamental para evaluar la efectividad del sistema de retrieval. El MRR proporciona una medida de la capacidad del sistema para devolver el documento correcto (en este caso, la película) en una posición alta de los resultados.

- Tiempo de Respuesta: El tiempo que toma al sistema procesar una consulta y devolver resultados. Es crítico para la experiencia del usuario, especialmente en aplicaciones interactivas.

- Carga del Sistema: Incluye la supervisión de la utilización de CPU, memoria y almacenamiento, así como el ancho de banda de red usado durante las operaciones de retrieval. Monitorear estos puede ayudar a anticipar necesidades de escalado y optimizar recursos.

- Tasa de Errores: El porcentaje de fallas o errores durante las operaciones de búsqueda y retrieval. Una alta tasa de errores puede indicar problemas subyacentes que necesitan resolverse para mantener la funcionalidad del sistema.

/ KPIs de Negocio

- Engagement del Usuario: Medido a través de métricas como el tiempo de sesión promedio, páginas vistas por sesión, y tasa de clicks (CTR) en las películas recomendadas. Un aumento en estas métricas puede indicar un mayor interés y satisfacción del usuario con las recomendaciones proporcionadas.

- Tasa de Conversión: En el contexto de una aplicación de streaming o similar, la tasa de conversión podría ser el porcentaje de recomendaciones que resultan en visualizaciones de películas. Una alta tasa de conversión indica una alineación exitosa entre las recomendaciones del sistema y las preferencias del usuario.

- Feedback de Usuarios: La cantidad y calidad del feedback proporcionado por los usuarios en relación con las recomendaciones. El análisis de estos datos puede ofrecer insights valiosos para ajustar y mejorar los algoritmos de recomendación.
